In [1]:
!pip install torch==0.4.0
!pip install --upgrade six

ERROR: Could not find a version that satisfies the requirement torch==0.4.0 (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2, 0.4.1, 0.4.1.post2, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1)
ERROR: No matching distribution found for torch==0.4.0
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is inc

In [2]:
!pip install transformers
!pip install boto3

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 46.8 MB/s 
     |████████████████████████████████| 596 kB 39.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 131 kB 5.4 MB/s 
     |████████████████████████████████| 8.0 MB 42.8 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 138 kB 46.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
r

In [3]:
!git clone https://github.com/microsoft/unilm.git

Cloning into 'unilm'...
remote: Enumerating objects: 3121, done.
remote: Counting objects: 100% (2272/2272), done.
remote: Compressing objects: 100% (1777/1777), done.
remote: Total 3121 (delta 942), reused 1585 (delta 419), pack-reused 849
Receiving objects: 100% (3121/3121), 5.22 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (1413/1413), done.


In [4]:
import os
os.chdir("unilm/unilm-v1/src")


In [5]:
ls

biunilm/  cnndm/  gigaword/  nn/  pytorch_pretrained_bert/  qg/  setup.py


In [6]:
import argparse
from dataclasses import dataclass
import json
import logging
import os
import random
import sys
import time
import warnings
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
import scipy.stats as st

import transformers
#import wandb
from tqdm.auto import tqdm, trange


logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s: %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [7]:
@dataclass
class CustomArguments(transformers.TrainingArguments):
    sample_train: int = 0
    sample_eval: int = 0
    num_choices: int = 0
    model_name_or_path: str = "asdf"  # this is no longer a TrainingArgument attribute
        
    # python dataclasses cannot have positional attributes in subclass,
    # so give all attributes defaults and then make sure they are changed
    def __post_init__(self):
        if not (self.sample_train * self.sample_eval * self.num_choices) or \
               self.model_name_or_path == "asdf":  # make sure none are still default value
            raise TypeError("__init__ missing required argument(s)")

def get_args():
    """ Set hyperparameters """
    args = CustomArguments(
        output_dir="checkpoint",
        model_name_or_path="roberta-base",
        overwrite_output_dir=True,
        do_train=False,  # Zero shot
        do_eval=True,
        per_device_eval_batch_size=2,
        learning_rate=1e-5,  # Should not matter because not training
        weight_decay=0.1,
        save_total_limit=2,
        seed=254,
        sample_train=200,
        sample_eval=-1,
        num_choices=5,
    )
    
    return args

In [8]:
def get_data(file_path, sample, num_choices):
    data_file = open(file_path, "r")
    logger.info("Reading QA instances from jsonl dataset at: %s", file_path)
    item_jsons = []
    item_ids = []
    questions = []
    choice_lists = []
    answer_ids = []
    for line in data_file:
        item_jsons.append(json.loads(line.strip()))

    if sample != -1:
        item_jsons = random.sample(item_jsons, sample)
        logger.info("Sampling %d examples", sample)

    for item_json in tqdm(item_jsons,total=len(item_jsons)):
        item_id = item_json["id"]

        question_text = item_json["question"]["stem"]

        choice_label_to_id = {}
        choice_text_list = []
        choice_context_list = []
        choice_label_list = []
        choice_annotations_list = []

        any_correct = False
        choice_id_correction = 0

        for choice_id, choice_item in enumerate(item_json["question"]["choices"]):
            choice_label = choice_item["label"]
            choice_label_to_id[choice_label] = choice_id - choice_id_correction
            choice_text = choice_item["text"]

            choice_text_list.append(choice_text)
            choice_label_list.append(choice_label)

            if item_json.get('answerKey') == choice_label:
                if any_correct:
                    raise ValueError("More than one correct answer found for {item_json}!")
                any_correct = True


        if not any_correct and 'answerKey' in item_json:
            raise ValueError("No correct answer found for {item_json}!")


        answer_id = choice_label_to_id.get(item_json.get("answerKey"))
        # Pad choices with empty strings if not right number
        if len(choice_text_list) != num_choices:
            choice_text_list = (choice_text_list + num_choices * [''])[:num_choices]
            choice_context_list = (choice_context_list + num_choices * [None])[:num_choices]
            if answer_id is not None and answer_id >= num_choices:
                logging.warning(f"Skipping question with more than {num_choices} answers: {item_json}")
                continue

        item_ids.append(item_id)
        questions.append(question_text)
        choice_lists.append(choice_text_list)
        answer_ids.append(answer_id)

    data_file.close()
    return questions, choice_lists, answer_ids

In [9]:
class BERTDataset(Dataset):
    
    def __init__(self, questions, choices, answer_ids, tokenizer):
        out = tokenizer(questions, max_length=25, padding="max_length")
        self.input_ids = out["input_ids"]
        self.token_type_ids = out["token_type_ids"]
        self.attention_mask = out["attention_mask"]
        self.questions = questions
        self.choices = choices
        self.answer_ids = answer_ids
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return {
            "input_ids": self.input_ids[i], 
            "attention_mask": self.attention_mask[i], 
            "token_type_ids": self.token_type_ids[i],
            "choice_list": self.choices[i], 
            "answer_id": self.answer_ids[i],
        }
    

class RoBERTaDataset(Dataset):
    
    def __init__(self, questions, choices, answer_ids, tokenizer):
#         if "t5" in tokenizer.name_or_path.lower():
#             questions = [question.replace('[MASK]', '') for question in questions]
#         else:
        questions = [question.replace('[MASK]', tokenizer.mask_token) for question in questions]
        out = tokenizer(questions, max_length=25, padding="max_length")
        self.input_ids = out["input_ids"]
        self.attention_mask = out["attention_mask"]
        self.questions = questions
        self.choices = choices
        self.answer_ids = answer_ids
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return {
            "input_ids": self.input_ids[i], 
            "attention_mask": self.attention_mask[i], 
            "choice_list": self.choices[i], 
            "answer_id": self.answer_ids[i],
        }

In [10]:
import os
import pandas as pd
os.getcwd()

'/content/unilm/unilm-v1/src'

In [12]:
#restart runtime
from biunilm import *
from pytorch_pretrained_bert.modeling import *
from pytorch_pretrained_bert.file_utils import *

In [13]:
!wget https://unilm.blob.core.windows.net/ckpt/unilm1-base-cased.bin
!wget https://unilm.blob.core.windows.net/ckpt/unilm1-large-cased.bin

--2021-10-18 13:47:16--  https://unilm.blob.core.windows.net/ckpt/unilm1-base-cased.bin
Resolving unilm.blob.core.windows.net (unilm.blob.core.windows.net)... 52.239.193.100
Connecting to unilm.blob.core.windows.net (unilm.blob.core.windows.net)|52.239.193.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217918554 (208M) [application/octet-stream]
Saving to: ‘unilm1-base-cased.bin’

unilm1-base-cased.b 100%[===================>] 207.82M  14.0MB/s    in 17s     

2021-10-18 13:47:33 (12.5 MB/s) - ‘unilm1-base-cased.bin’ saved [217918554/217918554]

--2021-10-18 13:47:33--  https://unilm.blob.core.windows.net/ckpt/unilm1-large-cased.bin
Resolving unilm.blob.core.windows.net (unilm.blob.core.windows.net)... 52.239.193.100
Connecting to unilm.blob.core.windows.net (unilm.blob.core.windows.net)|52.239.193.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 669422755 (638M) [application/octet-stream]
Saving to: ‘unilm1-large-cased.bin’



In [14]:
import math
import os
import logging
import shutil
import tempfile
import json
from urllib.parse import urlparse
from pathlib import Path
from typing import Optional, Tuple, Union, IO, Callable, Set
from hashlib import sha256
from functools import wraps

from tqdm import tqdm

import boto3
from botocore.exceptions import ClientError
import requests
args = get_args()
'''
"bert-base-uncased"
"bert-large-uncased"
'''

type_vocab_size = 6
args.model_name_or_path = "bert-large-cased"
transformers.set_seed(args.seed)

model = BertForPreTrainingLossMask.from_pretrained(args.model_name_or_path, num_rel=0, type_vocab_size=type_vocab_size).cuda()
model.load_state_dict(torch.load('unilm1-large-cased.bin', map_location=torch.device('cuda')))

tokenizer = transformers.BertTokenizer.from_pretrained(args.model_name_or_path)

10/18/2021 13:48:25: https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz not found in cache, downloading to /tmp/tmpkkecojnc
100%|██████████| 1242874899/1242874899 [00:35<00:00, 34618137.78B/s]
10/18/2021 13:49:01: copying /tmp/tmpkkecojnc to cache at /root/.pytorch_pretrained_bert/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
10/18/2021 13:49:05: creating metadata file for /root/.pytorch_pretrained_bert/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
10/18/2021 13:49:05: removing temp file /tmp/tmpkkecojnc
10/18/2021 13:49:06: loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz from cache at /root/.pytorch_pretrained_bert/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
10/1

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

10/18/2021 13:50:25: Attempting to release lock 140622681406608 on /root/.cache/huggingface/transformers/c9961ea5b7e8ad58701728c45f4d225f70b19aa59745121e5a96c8a44efca4c8.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock
10/18/2021 13:50:25: Lock 140622681406608 released on /root/.cache/huggingface/transformers/c9961ea5b7e8ad58701728c45f4d225f70b19aa59745121e5a96c8a44efca4c8.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock
10/18/2021 13:50:26: Attempting to acquire lock 140622680381520 on /root/.cache/huggingface/transformers/45d2aa048795efc7b12791662c188d5e3aa2f9ac54b2cf3f6e4d7bc6544e3d13.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock
10/18/2021 13:50:26: Lock 140622680381520 acquired on /root/.cache/huggingface/transformers/45d2aa048795efc7b12791662c188d5e3aa2f9ac54b2cf3f6e4d7bc6544e3d13.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

10/18/2021 13:50:26: Attempting to release lock 140622680381520 on /root/.cache/huggingface/transformers/45d2aa048795efc7b12791662c188d5e3aa2f9ac54b2cf3f6e4d7bc6544e3d13.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock
10/18/2021 13:50:26: Lock 140622680381520 released on /root/.cache/huggingface/transformers/45d2aa048795efc7b12791662c188d5e3aa2f9ac54b2cf3f6e4d7bc6544e3d13.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock
10/18/2021 13:50:26: Attempting to acquire lock 140622673056272 on /root/.cache/huggingface/transformers/75be22d7750034989358861e325977feda47740e1c3f8a4dc1cb73570aad843e.2b9a196704f2f183fe3f4b48d6e662dba8203fdcb3346bfa896831378edf6f97.lock
10/18/2021 13:50:26: Lock 140622673056272 acquired on /root/.cache/huggingface/transformers/75be22d7750034989358861e325977feda47740e1c3f8a4dc1cb73570aad843e.2b9a196704f2f183fe3f4b48d6e662dba8203fdcb3346bfa896831378edf6f97.lock


Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

10/18/2021 13:50:27: Attempting to release lock 140622673056272 on /root/.cache/huggingface/transformers/75be22d7750034989358861e325977feda47740e1c3f8a4dc1cb73570aad843e.2b9a196704f2f183fe3f4b48d6e662dba8203fdcb3346bfa896831378edf6f97.lock
10/18/2021 13:50:27: Lock 140622673056272 released on /root/.cache/huggingface/transformers/75be22d7750034989358861e325977feda47740e1c3f8a4dc1cb73570aad843e.2b9a196704f2f183fe3f4b48d6e662dba8203fdcb3346bfa896831378edf6f97.lock
10/18/2021 13:50:27: Attempting to acquire lock 140622629586192 on /root/.cache/huggingface/transformers/11ad22b0deaa199d15b331609ca5f60872a1a91473e9b40c115192dadb6d9a30.bdf0177a774dcff07681b2527b926c099e6563687c75a79f7469c7a7da7898c7.lock
10/18/2021 13:50:27: Lock 140622629586192 acquired on /root/.cache/huggingface/transformers/11ad22b0deaa199d15b331609ca5f60872a1a91473e9b40c115192dadb6d9a30.bdf0177a774dcff07681b2527b926c099e6563687c75a79f7469c7a7da7898c7.lock


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

10/18/2021 13:50:27: Attempting to release lock 140622629586192 on /root/.cache/huggingface/transformers/11ad22b0deaa199d15b331609ca5f60872a1a91473e9b40c115192dadb6d9a30.bdf0177a774dcff07681b2527b926c099e6563687c75a79f7469c7a7da7898c7.lock
10/18/2021 13:50:27: Lock 140622629586192 released on /root/.cache/huggingface/transformers/11ad22b0deaa199d15b331609ca5f60872a1a91473e9b40c115192dadb6d9a30.bdf0177a774dcff07681b2527b926c099e6563687c75a79f7469c7a7da7898c7.lock


In [18]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/antonym_synonym_negation/antonym_synonym_negation_dev.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/size_comparison/size_comparison_dev.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/compositional_comparison/compositional_comparison_dev.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_dev.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/coffee_cats_quantifiers/coffee_cats_quantifiers_dev.jsonl.gz
# !wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/conjunction/conjunction_filt4_dev.jsonl.gz
# !wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/hypernym_conjunction/hypernym_conjunction_dev.jsonl.gz
# !wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/composition/composition_v2_dev.jsonl.gz

--2021-10-18 13:56:49--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/antonym_synonym_negation/antonym_synonym_negation_dev.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.101.66
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.101.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17333 (17K) [binary/octet-stream]
Saving to: ‘antonym_synonym_negation_dev.jsonl.gz’

antonym_synonym_neg 100%[===================>]  16.93K  --.-KB/s    in 0.03s   

2021-10-18 13:56:49 (618 KB/s) - ‘antonym_synonym_negation_dev.jsonl.gz’ saved [17333/17333]

--2021-10-18 13:56:49--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/size_comparison/size_comparison_dev.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.101.66
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.101.66|:44

In [19]:
!gunzip antonym_synonym_negation_dev.jsonl.gz
!gunzip compositional_comparison_dev.jsonl.gz
!gunzip size_comparison_dev.jsonl.gz
!gunzip coffee_cats_quantifiers_dev.jsonl.gz
!gunzip number_comparison_age_compare_masked_dev.jsonl.gz
# !gunzip conjunction_filt4_dev.jsonl.gz
# !gunzip hypernym_conjunction_dev.jsonl.gz
# !gunzip composition_v2_dev.jsonl.gz

In [20]:
'''
Mask_task MC-MLM
"coffee_cats_quantifiers_dev.jsonl"   num_choices = 5 , Always-Never
"size_comparison_dev.jsonl" num_choices = 2 , Object comparison
"antonym_synonym_negation_dev.jsonl" num_choices = 2
"number_comparison_age_compare_masked_dev.jsonl"  num_choices = 2
"compositional_comparison_dev.jsonl" , args.num_choices = 3 , multihop composition

'''
train_path = "coffee_cats_quantifiers_dev.jsonl"
data  = train_path
args.num_choices = 5
eval_path = train_path
train_questions, train_choices, train_answer_ids = get_data(train_path, args.sample_train, args.num_choices)
eval_questions, eval_choices, eval_answer_ids = get_data(eval_path, args.sample_eval, args.num_choices)
AgeDataset = RoBERTaDataset if any(prefix in args.model_name_or_path.lower() for prefix in ("roberta", "bart", "distil", "electra", "t5")) else BERTDataset
train_dataset = AgeDataset(train_questions, train_choices, train_answer_ids, tokenizer)
#eval_dataset = AgeDataset(eval_questions, eval_choices, eval_answer_ids, tokenizer)

10/18/2021 13:57:16: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
10/18/2021 13:57:16: Sampling 200 examples
100%|██████████| 200/200 [00:00<00:00, 56891.20it/s]
10/18/2021 13:57:16: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
100%|██████████| 280/280 [00:00<00:00, 205747.22it/s]


In [21]:
def get_sentence_prob(input_ids, logits):
    # Multiplies together individual probabilities to get overall sentence probability
    logits = torch.nn.functional.softmax(logits, dim=2)
    probs = torch.gather(logits, 2, input_ids.unsqueeze(-1))
    probs = probs.squeeze(-1)
    probs = probs * 1e4  # product is zero otherwise
    #probs = torch.prod(probs, dim=1)
    probs = torch.sum(torch.log(probs), dim=1)
    return probs

In [22]:
def evaluate(args, model, tokenizer, eval_dataset):
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.per_device_eval_batch_size)

    logger.info(f"***** Running evaluation  *****")
    logger.info(f"  Num examples = {len(eval_dataset)}")
    logger.info(f"  Batch size = {args.eval_batch_size}")
    eval_dataloader = tqdm(eval_dataloader, desc="Evaluating")
    
    print(tokenizer.mask_token)
    MASK_ID = tokenizer.encode(tokenizer.mask_token, add_special_tokens=False)
    assert len(MASK_ID) == 1
    MASK_ID = MASK_ID[0]
    
    all_answers = []
    all_preds = []
    for batch in eval_dataloader:
        model.eval()
        
        # batch["choice_list"] is [num_choices, batch_size]
        for i in range(len(batch["choice_list"][0])):
            all_answers.append(batch["choice_list"][batch["answer_id"][i]][i])
        
        choice_lists = batch.pop("choice_list")
        
        del batch["answer_id"] 
        for key in batch:
            batch[key] = torch.stack(batch[key], dim=-1).cuda()
       
        for i in range(len(batch["input_ids"])):
          question = batch["input_ids"][i]
        
        with torch.no_grad():
           
            outputs = model(**batch)
            logits = outputs[0]
            choice_ids = []

            for i, logit in enumerate(logits):  # Assuming all are single tokens
              #print(batch["input_ids"][i])
              #print(tokenizer.convert_ids_to_tokens(batch["input_ids"][i]))
              MASK_INDEX = batch["input_ids"][i].tolist().index(MASK_ID)
              #print(MASK_INDEX)
              choice_ids = torch.tensor([tokenizer.encode(" " + choice_lists[j][i], add_special_tokens=False)[0] for j in range(len(choice_lists))])
              choice_ids = choice_ids.cuda()
              #print("choices_ids after logits ", choice_ids)
              probs = logit[MASK_INDEX].index_select(0, choice_ids).cuda()
              #print("probabilities ", probs)
              max_ind = torch.argmax(probs)
              #print("max_index  ", max_ind)
              all_preds.append(choice_lists[max_ind][i])

    return all_answers, all_preds

In [23]:
def evaluate_taskwithmask(args, model, tokenizer, eval_dataset, data_path):
    
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.per_device_eval_batch_size)

    logger.info(f"***** Running evaluation  *****")
    logger.info(f"  Num examples = {len(eval_dataset)}")
    logger.info(f"  Batch size = {args.eval_batch_size}")
    eval_dataloader = tqdm(eval_dataloader, desc="Evaluating")
    
    #encoding all the labels
    label_dict = {}
    label_encodings = {}
    list_of_labels = eval_dataset[0]['choice_list']
    i=0
    for label in list_of_labels:
      label_dict[label] = i
      label1 = " "+ label
      label_encodings[label1] = tokenizer.encode(label1, add_special_tokens=False)[0]
      i+=1

    label_id_encoding_map = dict(zip(label_dict.values(),label_encodings.values()))

    all_answers = []
    all_preds = []
    first_age = []
    second_age = []
    first_object = []
    second_object = []
    
    #create list of true answers =  all_answers 
    for batch in eval_dataloader:
        original_batch = batch          
        model.eval()
        for i in range(len(batch["answer_id"])):
            true_label_id = batch["answer_id"][i]
            actual_label = batch["choice_list"][true_label_id][i]
            label_id_to_append = label_dict[actual_label]
            all_answers.append(label_id_to_append)
           

        del batch["choice_list"] 
        for key in batch:
            if key != "answer_id":
                batch[key] = torch.stack(batch[key], dim=-1)

            batch[key] = batch[key].cuda()
      
        
        answer_ids = batch.pop("answer_id")
        label_encoding_list = list(label_encodings.values())
        no_of_labels = len(label_encoding_list)

        #create the list for age1, age2 for age task and list of objects for object comparison task
        if data_path == "number_comparison_age_compare_masked_dev.jsonl":
            age1 = tokenizer.decode(batch["input_ids"][:, 1]).split(" ")
            age2 = tokenizer.decode(batch["input_ids"][:, 11]).split(" ")
            age1 = age1[1:]
            age2 = age2[1:]
            first_age.extend(age1)
            second_age.extend(age2)


        with torch.no_grad():
            #generate probablities for all the labels
            
            list_of_mask_index = []
           
            for i in range(len(batch["input_ids"])):
                  question = batch["input_ids"][i]
                  MASK_INDEX = (question==tokenizer.mask_token_id).nonzero().item()
                  batch["input_ids"][i, MASK_INDEX] =  label_id_encoding_map[0]
                  list_of_mask_index.append(MASK_INDEX)
            
            outputs = model(**batch)
            logits = outputs[0]
            id0_prob = get_sentence_prob(batch["input_ids"], logits)
           
            m=0
            for i in range(len(batch["input_ids"])):
                question = batch["input_ids"][i]
                MASK_INDEX = list_of_mask_index[m]
                batch["input_ids"][i, MASK_INDEX] =  label_id_encoding_map[1]
                m +=1
      
            outputs = model(**batch)
            logits = outputs[0]
            id1_prob = get_sentence_prob(batch["input_ids"], logits)
            
            if no_of_labels==3:
              m=0
              for i in range(len(batch["input_ids"])):
                  question = batch["input_ids"][i]
                  MASK_INDEX = list_of_mask_index[m]
                  batch["input_ids"][i, MASK_INDEX] =  label_id_encoding_map[2]
                  m +=1
                  
              outputs = model(**batch)
              logits = outputs[0]
              id2_prob = get_sentence_prob(batch["input_ids"], logits)

            if no_of_labels==5:
                m=0
                for i in range(len(batch["input_ids"])):
                    question = batch["input_ids"][i]
                    MASK_INDEX = list_of_mask_index[m]
                    batch["input_ids"][i, MASK_INDEX] =  label_id_encoding_map[2]
                    m +=1
                
                outputs = model(**batch)
                logits = outputs[0]
                id2_prob = get_sentence_prob(batch["input_ids"], logits)

                m=0
                for i in range(len(batch["input_ids"])):
                    question = batch["input_ids"][i]
                    MASK_INDEX = list_of_mask_index[m]
                    batch["input_ids"][i, MASK_INDEX] =  label_id_encoding_map[3]
                    m +=1
                outputs = model(**batch)
                logits = outputs[0]
                id3_prob = get_sentence_prob(batch["input_ids"], logits)

                m=0
                for i in range(len(batch["input_ids"])):
                    question = batch["input_ids"][i]
                    MASK_INDEX = list_of_mask_index[m]
                    batch["input_ids"][i, MASK_INDEX] =  label_id_encoding_map[4]
                    m +=1
                  
                outputs = model(**batch)
                logits = outputs[0]
                id4_prob = get_sentence_prob(batch["input_ids"], logits)
  
        batch_size = len(batch["input_ids"])
        #create all_preds
        if no_of_labels ==2:
          test_pred = torch.gt(id0_prob, id0_prob)
          id0_prob = torch.reshape(id0_prob, (batch_size, 1))
          id1_prob = torch.reshape(id1_prob, (batch_size, 1))
          combine_prob = torch.cat((id0_prob, id1_prob), dim=1)
          preds = list(torch.argmax(combine_prob, dim=1))
          all_preds.extend(preds)
        elif no_of_labels ==3:
          id0_prob = torch.reshape(id0_prob, (batch_size, 1))
          id1_prob = torch.reshape(id1_prob, (batch_size, 1))
          id2_prob = torch.reshape(id2_prob, (batch_size, 1))
          combine_prob = torch.cat((id0_prob, id1_prob, id2_prob), dim=1)
          preds = list(torch.argmax(combine_prob, dim=1))
          all_preds.extend(preds)
        elif no_of_labels ==5:
          id0_prob = torch.reshape(id0_prob, (batch_size, 1))
          id1_prob = torch.reshape(id1_prob,(batch_size, 1))
          id2_prob = torch.reshape(id2_prob, (batch_size, 1))
          id3_prob = torch.reshape(id3_prob, (batch_size, 1))
          id4_prob = torch.reshape(id4_prob, (batch_size, 1))
          combine_prob = torch.cat((id0_prob, id1_prob, id2_prob, id3_prob, id4_prob), dim=1)
          preds = list(torch.argmax(combine_prob, dim=1))
          all_preds.extend(preds)
    return all_answers, all_preds

In [24]:
#choose random 80% of the dataset and run evaluation for 5 cycles
accuracy = []
for i in range(5):
  eval_questions, eval_choices, eval_answer_ids = get_data(data, args.sample_eval, args.num_choices)
  combined_dataset = {'que': eval_questions, 'choices': eval_choices, 'ids': eval_answer_ids, }
  combined_dataset = pd.DataFrame(data=combined_dataset)
  sampled_dataset = combined_dataset.sample(frac = 0.8)
  eval_questions = list(sampled_dataset['que'])
  eval_choices = list(sampled_dataset['choices'])
  eval_answer_ids = list(sampled_dataset['ids'])

  eval_dataset = AgeDataset(eval_questions, eval_choices, eval_answer_ids, tokenizer)
  all_answers, all_preds = evaluate(args, model, tokenizer, eval_dataset)
  a = 0
  b = 0
  for i in range(len(all_answers)):
    if all_preds[i] != -1:
        b += 1
        if all_preds[i] == all_answers[i]:
            a += 1
  current_acc = a/b
  accuracy.append(current_acc)

10/18/2021 13:57:32: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
100%|██████████| 280/280 [00:00<00:00, 61096.93it/s]
10/18/2021 13:57:32: ***** Running evaluation  *****
10/18/2021 13:57:32:   Num examples = 224
10/18/2021 13:57:32:   Batch size = 2
Evaluating:   0%|          | 0/112 [00:00<?, ?it/s]

[MASK]


Evaluating: 100%|██████████| 112/112 [00:07<00:00, 15.13it/s]
10/18/2021 13:57:40: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
100%|██████████| 280/280 [00:00<00:00, 120538.35it/s]
10/18/2021 13:57:40: ***** Running evaluation  *****
10/18/2021 13:57:40:   Num examples = 224
10/18/2021 13:57:40:   Batch size = 2
Evaluating:   2%|▏         | 2/112 [00:00<00:06, 15.90it/s]

[MASK]


Evaluating: 100%|██████████| 112/112 [00:06<00:00, 16.04it/s]
10/18/2021 13:57:47: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
100%|██████████| 280/280 [00:00<00:00, 112415.54it/s]
10/18/2021 13:57:47: ***** Running evaluation  *****
10/18/2021 13:57:47:   Num examples = 224
10/18/2021 13:57:47:   Batch size = 2
Evaluating:   2%|▏         | 2/112 [00:00<00:07, 15.48it/s]

...

Evaluating: 100%|██████████| 112/112 [00:06<00:00, 16.05it/s]
10/18/2021 13:57:54: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
100%|██████████| 280/280 [00:00<00:00, 159783.01it/s]
10/18/2021 13:57:54: ***** Running evaluation  *****
10/18/2021 13:57:54:   Num examples = 224
10/18/2021 13:57:54:   Batch size = 2
Evaluating:   2%|▏         | 2/112 [00:00<00:06, 16.29it/s]

[MASK]


Evaluating: 100%|██████████| 112/112 [00:06<00:00, 16.05it/s]
10/18/2021 13:58:01: Reading QA instances from jsonl dataset at: coffee_cats_quantifiers_dev.jsonl
100%|██████████| 280/280 [00:00<00:00, 102639.85it/s]
10/18/2021 13:58:01: ***** Running evaluation  *****
10/18/2021 13:58:01:   Num examples = 224
10/18/2021 13:58:01:   Batch size = 2
Evaluating:   2%|▏         | 2/112 [00:00<00:07, 14.19it/s]

[MASK]


Evaluating: 100%|██████████| 112/112 [00:06<00:00, 16.04it/s]


In [25]:
accuracy

[0.16964285714285715, 0.1875, 0.21428571428571427, 0.1875, 0.20089285714285715]

In [26]:
#Number of samples = 5
#create 95% confidence interval for population mean weight
mini, maxi = st.t.interval(alpha=0.95, df=len(accuracy)-1, loc=np.mean(accuracy), scale=st.sem(accuracy))
accuracy = np.array(accuracy)

In [27]:
print("The accuracy is of {} for {} task is {} +- {}".format(args.model_name_or_path, train_path, accuracy.mean()*100 ,-1*accuracy.mean()*100+maxi*100))

The accuracy is of bert-large-cased for coffee_cats_quantifiers_dev.jsonl task is 19.196428571428573 +- 2.0740541387367806


In [28]:
#unilm-base

#The accuracy is of unilm-base for coffee_cats_quantifiers_dev.jsonl task 
#[0.16517857142857142,
 0.14732142857142858,
 0.16517857142857142,
 0.16071428571428573,
 0.13839285714285715]
 # 15.535714285714283 +- 1.4873813063559655

#The accuracy is of unilm-base for size_comparison_dev.jsonl task is 
#[0.485, 0.4875, 0.475, 0.4825, 0.4625]
# 47.849999999999994 +- 1.2532505385071673


#The accuracy is of unilm-base for antonym_synonym_negation_dev.jsonl task is 
#[0.4425, 0.4275, 0.4325, 0.4375, 0.4325]
#43.45 +- 0.707857388491135

#The accuracy is of unilm-base for compositional_comparison_dev.jsonl task is 
#[0.35, 0.3525, 0.3375, 0.3525, 0.35]
#34.85 +- 0.779137975262671

#The accuracy is of unilm-base for number_comparison_age_compare_masked_dev.jsonl task is 
#[0.4875, 0.48, 0.4725, 0.46, 0.4925]
#47.85 +- 1.5919226865776608

#5parts
#[0.4075, 0.4525, 0.515, 0.5475, 0.6075]
# 50.6 +- 9.751213563067473





IndentationError: ignored

In [ ]:
#unilm-large

#The accuracy is of unilm-base for coffee_cats_quantifiers_dev.jsonl task 
#[0.16964285714285715, 0.1875, 0.21428571428571427, 0.1875, 0.20089285714285715]
#19.196428571428573 +- 2.0740541387367806

#The accuracy is of unilm-base for size_comparison_dev.jsonl task 
#[0.5975, 0.605, 0.61, 0.6275, 0.6175]
# 61.150000000000006 +- 1.4326295320684181

#The accuracy is of unilm-base for antonym_synonym_negation_dev.jsonl task is 
#[0.5075, 0.5175, 0.5025, 0.51, 0.5025]
# 50.79999999999999 +- 0.7729296056037143

#The accuracy is of unilm-base for compositional_comparison_dev.jsonl task is
#[0.345, 0.3175, 0.3325, 0.33, 0.33]
#33.1 +- 1.21419883853153

#The accuracy is of unilm-base for number_comparison_age_compare_masked_dev.jsonl task is 
#[0.4875, 0.48, 0.4725, 0.46, 0.4925]
# 47.85 +- 1.5919226865776608
#5parts
#[0.375, 0.42, 0.48, 0.5325, 0.5725]
# 47.599999999999994 +- 9.983144562685077


In [ ]:
if train_path == 'number_comparison_age_compare_masked_dev.jsonl':
  all_answers, all_preds = evaluate_taskwithmask(args, model, tokenizer, eval_dataset, train_path)
  #only for number comparison
  #run the evaluation for five times to capture variation in the results. Dataset is divided into 5 parts and one part is dropped with each evaluation cycle.

  accuracy = []
  for i in range(5):
    eval_questions, eval_choices, eval_answer_ids = get_data(train_path, args.sample_eval, args.num_choices)
    total_items = len(eval_questions)
    n = int(total_items/5)
    if i==0:
      eval_questions = eval_questions[n:]
      eval_choices = eval_choices[n:]
      eval_answer_ids = eval_answer_ids[n:] 
    elif i==4:
      eval_questions = eval_questions[:4*n]
      eval_choices = eval_choices[:4*n]
      eval_answer_ids = eval_answer_ids[:4*n]
      
    else:
      eval_questions = eval_questions[:i*n] + eval_questions[(i+1)*n:]
      eval_choices = eval_choices[:i*n] + eval_choices[(i+1)*n:]
      eval_answer_ids = eval_answer_ids[:i*n] + eval_answer_ids[(i+1)*n:]   

    eval_dataset = AgeDataset(eval_questions, eval_choices, eval_answer_ids, tokenizer)
    print(len(eval_dataset))
    #if train_path == "hypernym_conjunction_dev.jsonl" or train_path =="composition_v2_dev.jsonl" or train_path=="conjunction_filt4_dev.jsonl":
    #all_answers, all_preds = evaluateqa(args, model, tokenizer, eval_dataset, train_path)
    #else:
    all_answers, all_preds = evaluateqa(args, model, tokenizer, eval_dataset)
    a = 0
    b = 0
    for i in range(len(all_answers)):
      if all_preds[i] != -1:
          b += 1
          if all_preds[i] == all_answers[i]:
              a += 1
    current_acc = a/b
    accuracy.append(current_acc)



In [ ]:
accuracy

In [ ]:
#Number of samples = 5
#create 95% confidence interval for population mean weight
mini, maxi = st.t.interval(alpha=0.95, df=len(accuracy)-1, loc=np.mean(accuracy), scale=st.sem(accuracy))
accuracy = np.array(accuracy)

In [ ]:
print("The accuracy is of {} for {} task is {} +- {}".format(args.model_name_or_path, train_path, accuracy.mean()*100 ,-1*accuracy.mean()*100+maxi*100))

In [ ]:
#5 parts
#unilm-base
#The accuracy is of unilm-base for compositional_comparison_dev.jsonl task is 34.8 +- 0.8888961473496906
#[0.3475, 0.3375, 0.3575, 0.3475, 0.35]

#The accuracy is of unilm-base for antonym_synonym_negation_dev.jsonl task is 43.599999999999994 +- 1.291121607380866
#[0.44 0.4275 0.4525 0.43   0.43  ]

#The accuracy is of unilm-base for number_comparison_age_compare_masked_dev.jsonl task is 49.400000000000006 +- 9.75121356306746
#[0.5925 0.5475 0.485  0.4525 0.3925]

#The accuracy is of unilm-base for size_comparison_dev.jsonl task is 48.0 +- 2.624805446803734
#[0.5175, 0.47, 0.4675, 0.47, 0.475]

#The accuracy is of unilm-base for coffee_cats_quantifiers_dev.jsonl task is 16.07142857142857 +- 0.7839187795402545
#
[0.15178571428571427, 0.16964285714285715, 0.16071428571428573, 0.16071428571428573, 0.16071428571428573]

In [ ]:
#5 parts
#unilm-large

#The accuracy is of unilm-large for compositional_comparison_dev.jsonl task is 33.599999999999994 +- 0.7146312977467488
#[0.3375, 0.33, 0.345, 0.3325, 0.335]

#The accuracy is of unilm-large for antonym_synonym_negation_dev.jsonl task is 51.19999999999999 +- 1.3777715440925036
#[0.52, 0.505, 0.525, 0.4975, 0.5125]

#The accuracy is of unilm-large for number_comparison_age_compare_masked_dev.jsonl task is 47.599999999999994 +- 9.98314456268507
#[0.375, 0.42, 0.48, 0.5325, 0.5725]

#The accuracy is of unilm-large for size_comparison_dev.jsonl task is 61.40000000000001 +- 1.8046893183785642
#[0.6025, 0.6275, 0.5975, 0.63, 0.6125]

#The accuracy is of unilm-large for coffee_cats_quantifiers_dev.jsonl task is 18.928571428571427 +- 1.3349655776296
#[0.19196428571428573, 0.19642857142857142, 0.17410714285714285, 0.18303571428571427, 0.20089285714285715]